### Importing required libraries

In [1]:
from src.models.dynamics_models import BidomainModel
from src.models.cell_models import *
from src.utils import *
import ufl

### Defining a domain, cell model and the whole model and visualizing initial data

In [2]:
domain = right_ventricle()
cell_model = ReparametrizedFitzHughNagumo(domain)
x = ufl.SpatialCoordinate(domain)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 10%] Meshing curve 2 (Circle)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 50%] Meshing curve 6 (Ellipse)
Info    : [ 60%] Meshing curve 7 (Ellipse)
Info    : [ 80%] Meshing curve 9 (Ellipse)
Info    : [ 90%] Meshing curve 10 (Ellipse)
Info    : [100%] Meshing curve 11 (Line)
Info    : Done meshing 1D (Wall 0.0139119s, CPU 0.014419s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 2 (Surface of Revolution, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 3 (Surface of Revolution, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.243904s, CPU 0.236511s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 4826 nodes...


In [3]:
class Model(BidomainModel):
    def initial_V_m(self):
        def value(x):
            return cell_model.V_REST + 25 * np.exp(
                -((x[0] - 0) ** 2) + -((x[1] - 0) ** 2) - (x[2] + 8) ** 2
            )

        self.V_m_n.interpolate(value)

    def conductivity(
        self,
        longitudinal_fibres=[
            x[1] / (x[0] ** 2 + x[1] ** 2),
            -x[0] / (x[0] ** 2 + x[1] ** 2),
        ],
        transversal_fibres=[0, 0],
    ):
        return super().conductivity(longitudinal_fibres, transversal_fibres)

    def ischemia(self):
        return (
            lambda x: (x[0] + 1.5) ** 2 + (x[1] - 0) ** 2 + (x[2] + 4) ** 2 < 1**2,
            self.M_i / 10,
            self.M_e / 10,
        )


model = Model(domain, cell_model)

In [4]:
model.plot_initial_V_m(show_mesh=False, shadow=True)

Widget(value='<iframe src="http://localhost:35413/index.html?ui=P_0x7f2e584fa710_0&reconnect=auto" class="pyvi…

In [5]:
def plot_function(
    function: fem.Function,
    function_name: str = "function",
    camera_direction: list[float] = [1, 1, 1],
    zoom: float = 1.0,
    shadow: bool = False,
    show_mesh: bool = True,
    save_to: str | None = None,
):
    """Plot a dolfinx fem Function."""
    # Create a pyvista mesh and attach the values of u
    grid = pyvista.UnstructuredGrid(*vtk_mesh(function.function_space))
    grid.point_data["function"] = function.x.array
    grid.set_active_scalars("function")

    # Create a light to illuminate a mesh
    light = pyvista.Light(position=[3.0, 3.0, 0.0], intensity=0.2)

    # We visualize the data
    plotter = pyvista.Plotter()
    plotter.add_text(
        f"{function_name}", position="upper_edge", font_size=14, color="black"
    )
    plotter.add_mesh(grid, show_edges=show_mesh, lighting=shadow, cmap="coolwarm")
    plotter.view_vector(camera_direction)
    plotter.add_light(light)
    plotter.camera.zoom(zoom)
    if save_to is not None:
        plotter.save_graphic(save_to)
    plotter.show()


plot_function(model.V_m_n, show_mesh=False, shadow=None)

Widget(value='<iframe src="http://localhost:39623/index.html?ui=P_0x7f7d16f8af50_1&reconnect=auto" class="pyvi…

In [6]:
model.plot_ischemia(show_mesh=False, shadow=True)

Widget(value='<iframe src="http://localhost:35413/index.html?ui=P_0x7f2e4b2886d0_2&reconnect=auto" class="pyvi…

In [ ]:
x_c, y_c = 0, 0
plot_vector_field(
    domain,
    lambda x: [
        (x[1] - y_c) / ((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2 + 1),
        -(x[0] - x_c) / ((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2 + 1),
        0,
    ],
    tolerance=0.03,
    factor=1,
)

Widget(value='<iframe src="http://localhost:44639/index.html?ui=P_0x7f6a9fc8c7d0_2&reconnect=auto" class="pyvi…

In [ ]:
žnj

NameError: name 'žnj' is not defined

ERROR:wslink.protocol:Exception raised
ERROR:wslink.protocol:KeyError('8a1c475b7218b56ead871f6b0892c0af_19298d')
ERROR:wslink.protocol:Traceback (most recent call last):
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/wslink/protocol.py", line 332, in onMessage
    results = func(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/trame_vtk/modules/vtk/protocols/local_rendering.py", line 33, in get_array
    self.context.get_cached_data_array(data_hash, binary)
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/trame_vtk/modules/vtk/serializers/synchronization_context.py", line 30, in get_cached_data_array
    cache_obj = self.data_array_cache[p_md5]
                ~~~~~~~~~~~~~~~~~~~~~^^^^^^^
KeyError: '8a1c475b7218b56ead871f6b0892c0af_19298d'



### Solving equations with given parameters

In [ ]:
model.solve(
    T=1000,
    steps=1000,
    save_to="demo.mp4",
    signal_point=[0.0, -2.0, -4.0],
)

### Plotting a final plot of transmembrane potential

In [ ]:
plot_function(model.V_m_n, "V_m", show_mesh=False)

### Plotting a transmembrane potential in a given signal point

In [ ]:
model.plot_signal()

### Plotting a transmembrane potential solution of a cell model

In [ ]:
cell_model.visualize(1000, -84)